In [1]:
%load_ext autoreload
%autoreload 2

# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd

# Otros objetivos
# -----------------------------------------------------------------------
import math

# Gráficos
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
import plotly_express as px

# traigo mis funciones
import sys
sys.path.append('../../') #pongo los 2 puntos para que salga de la carpeta
import src.funciones as f
import src.soporte_prep as spp
import src.soporte_encoding as se
# Métodos estadísticos
# -----------------------------------------------------------------------
from scipy.stats import zscore # para calcular el z-score
from sklearn.neighbors import LocalOutlierFactor # para detectar outliers usando el método LOF
from sklearn.ensemble import IsolationForest # para detectar outliers usando el metodo IF
from sklearn.neighbors import NearestNeighbors # para calcular la epsilon

# Para generar combinaciones de listas
# -----------------------------------------------------------------------
from itertools import product
from itertools import combinations
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder, TargetEncoder # para poder aplicar los métodos de OneHot, Ordinal,  Label y Target Encoder 

# Gestionar warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
from category_encoders import TargetEncoder

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_pickle('../../Datos_Mod2/df_estandarizado.pkl')

In [3]:
df.shape

(1568, 25)

In [4]:
df.head()

,Age_robust,Attrition,BusinessTravel,Department,DistanceFromHome_robust,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome_robust,NumCompaniesWorked_robust,PercentSalaryHike_robust,StockOptionLevel,TotalWorkingYears_robust,TrainingTimesLastYear,YearsAtCompany_robust,YearsSinceLastPromotion_robust,YearsWithCurrManager_robust,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,1.153846,0,Travel_Rarely,Sales,-0.083333,2,Life Sciences,Female,1,Healthcare Representative,Married,1.512817,-0.333333,-0.500000,0,-1.000000,6,-0.666667,-0.333333,-0.6,3.0,4.0,2.0,3,3
1,-0.384615,1,Travel_Frequently,Research & Development,0.250000,1,Life Sciences,Female,1,Research Scientist,Single,-0.130517,-0.666667,1.500000,1,-0.444444,3,0.000000,0.000000,0.2,3.0,2.0,4.0,2,4
2,-0.307692,0,Travel_Frequently,Research & Development,0.833333,4,Other,Male,4,Sales Executive,Married,2.656358,-0.333333,0.166667,3,-0.555556,2,0.000000,-0.333333,0.0,2.0,2.0,1.0,3,3
3,0.153846,0,Non-Travel,Research & Development,-0.416667,5,Life Sciences,Male,3,Human Resources,Married,0.630126,0.333333,-0.500000,3,0.333333,5,0.500000,2.000000,0.4,4.0,4.0,3.0,2,3
4,-0.307692,0,Travel_Rarely,Research & Development,0.250000,1,Medical,Male,1,Sales Executive,Single,-0.470523,0.666667,-0.333333,2,-0.111111,2,0.166667,-0.333333,0.2,4.0,1.0,3.0,3,3


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1568 entries, 0 to 1572
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   Age_robust                      1568 non-null   float64 
 1   Attrition                       1568 non-null   int64   
 2   BusinessTravel                  1568 non-null   object  
 3   Department                      1568 non-null   object  
 4   DistanceFromHome_robust         1568 non-null   float64 
 5   Education                       1568 non-null   category
 6   EducationField                  1568 non-null   object  
 7   Gender                          1568 non-null   object  
 8   JobLevel                        1568 non-null   category
 9   JobRole                         1568 non-null   object  
 10  MaritalStatus                   1568 non-null   object  
 11  MonthlyIncome_robust            1568 non-null   float64 
 12  NumCompaniesWorked_robust

In [6]:
lista_col_cat = df.select_dtypes(include=['object', 'category']).columns.tolist()

In [7]:
lista_col_num = df.select_dtypes(include=[int, float]).columns.tolist()

In [8]:
len(lista_col_cat)

15

In [9]:
len(lista_col_num)

10

In [10]:
df['EnvironmentSatisfaction'] = df['EnvironmentSatisfaction'].astype(str)
df['JobSatisfaction'] = df['JobSatisfaction'].astype(str)
df['WorkLifeBalance'] = df['WorkLifeBalance'].astype(str)

In [11]:
lista_ord, lista_des = f.detectar_orden_cat(df, lista_cat=df.select_dtypes(include=['object', 'category']), var_respuesta='Attrition')

Estamos evaluando la variable BUSINESSTRAVEL


Attrition,0,1
BusinessTravel,,
Non-Travel,145,12
Travel_Frequently,219,77
Travel_Rarely,952,163


Sí tiene orden la variable BusinessTravel
Estamos evaluando la variable DEPARTMENT


Attrition,0,1
Department,,
Human Resources,44,22
Research & Development,865,160
Sales,407,70


Sí tiene orden la variable Department
Estamos evaluando la variable EDUCATION


...

Sí tiene orden la variable WorkLifeBalance
Estamos evaluando la variable JOBINVOLVEMENT


Attrition,0,1
JobInvolvement,,
1,71,19
2,330,62
3,788,144
4,127,27


La variable JobInvolvement no tiene orden
Estamos evaluando la variable PERFORMANCERATING


Attrition,0,1
PerformanceRating,,
3,1120,210
4,196,42


La variable PerformanceRating no tiene orden


In [12]:
lista_ord

['BusinessTravel',
 'Department',
 'EducationField',
 'MaritalStatus',
 'EnvironmentSatisfaction',
 'JobSatisfaction',
 'WorkLifeBalance']

In [13]:
lista_des

['Education',
 'Gender',
 'JobLevel',
 'JobRole',
 'StockOptionLevel',
 'TrainingTimesLastYear',
 'JobInvolvement',
 'PerformanceRating']

In [14]:
diccionario_encoding = { "target": lista_ord, 
                        "onehot": lista_des} 
# En caso de que queramos aplicar el método solo a una columna
encoding = se.Encoding(df, variable_respuesta="Attrition", diccionario_encoding=diccionario_encoding)
df_resultado = encoding.one_hot_encoding()
df_resultado = encoding.target_encoding()

['BusinessTravel', 'Department', 'EducationField', 'MaritalStatus', 'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance']


In [15]:
df_resultado.head()

,Age_robust,Attrition,BusinessTravel,Department,DistanceFromHome_robust,EducationField,MaritalStatus,MonthlyIncome_robust,NumCompaniesWorked_robust,PercentSalaryHike_robust,TotalWorkingYears_robust,YearsAtCompany_robust,YearsSinceLastPromotion_robust,YearsWithCurrManager_robust,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Education_1,Education_2,Education_3,Education_4,Education_5,Gender_Female,Gender_Male,JobLevel_1,JobLevel_2,JobLevel_3,JobLevel_4,JobLevel_5,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,StockOptionLevel_0,StockOptionLevel_1,StockOptionLevel_2,StockOptionLevel_3,TrainingTimesLastYear_0,TrainingTimesLastYear_1,TrainingTimesLastYear_2,TrainingTimesLastYear_3,TrainingTimesLastYear_4,TrainingTimesLastYear_5,TrainingTimesLastYear_6,JobInvolvement_1,JobInvolvement_2,JobInvolvement_3,JobInvolvement_4,PerformanceRating_3,PerformanceRating_4
0,1.153846,0,0.146188,0.146751,-0.083333,0.165644,0.118384,1.512817,-0.333333,-0.500000,-1.000000,-0.666667,-0.333333,-0.6,0.139004,0.112474,0.170868,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1,-0.384615,1,0.260135,0.156098,0.250000,0.165644,0.263158,-0.130517,-0.666667,1.500000,-0.444444,0.000000,0.000000,0.2,0.139004,0.166667,0.179487,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,-0.307692,0,0.260135,0.156098,0.833333,0.111156,0.118384,2.656358,-0.333333,0.166667,-0.555556,0.000000,-0.333333,0.0,0.149007,0.166667,0.325000,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.153846,0,0.076433,0.156098,-0.416667,0.165644,0.118384,0.630126,0.333333,-0.500000,0.333333,0.500000,2.000000,0.4,0.134199,0.112474,0.140875,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,-0.307692,0,0.146188,0.156098,0.250000,0.166324,0.263158,-0.470523,0.666667,-0.333333,-0.111111,0.166667,-0.333333,0.2,0.134199,0.230000,0.140875,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [16]:
df_resultado.columns

Index(['Age_robust', 'Attrition', 'BusinessTravel', 'Department',
       'DistanceFromHome_robust', 'EducationField', 'MaritalStatus',
       'MonthlyIncome_robust', 'NumCompaniesWorked_robust',
       'PercentSalaryHike_robust', 'TotalWorkingYears_robust',
       'YearsAtCompany_robust', 'YearsSinceLastPromotion_robust',
       'YearsWithCurrManager_robust', 'EnvironmentSatisfaction',
       'JobSatisfaction', 'WorkLifeBalance', 'Education_1', 'Education_2',
       'Education_3', 'Education_4', 'Education_5', 'Gender_Female',
       'Gender_Male', 'JobLevel_1', 'JobLevel_2', 'JobLevel_3', 'JobLevel_4',
       'JobLevel_5', 'JobRole_Healthcare Representative',
       'JobRole_Human Resources', 'JobRole_Laboratory Technician',
       'JobRole_Manager', 'JobRole_Manufacturing Director',
       'JobRole_Research Director', 'JobRole_Research Scientist',
       'JobRole_Sales Executive', 'JobRole_Sales Representative',
       'StockOptionLevel_0', 'StockOptionLevel_1', 'StockOptionLevel_2',

In [17]:
df_resultado.to_pickle('../../Datos_Mod2/df_encoded.pkl')